# import modules

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


2024-07-28 03:48:21.272807: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 03:48:21.275525: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 03:48:21.284326: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 03:48:21.301560: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 03:48:21.301594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 03:48:21.313563: I tensorflow/core/platform/cpu_feature_guard.cc:

# import data set

In [3]:
#loading data
file_1="/workspaces/State-of-Health-Estimation-Battery/Dataset/B0005_dis_dataset.csv"
file_2="/workspaces/State-of-Health-Estimation-Battery/Dataset/B0006_dis_dataset.csv"
dataset_1=pd.read_csv(file_1)
dataset_2=pd.read_csv(file_2)

C_max=2.05

#renaming columns for simplicity
dataset_1.columns=['cycle',"capacity",'voltage', 'current','temperature', 'current_load', 'voltage_load',"time"]
dataset_1["current_load"]= abs(dataset_1["current_load"])

#removing ideling / starting and ending current level data  
df_1=dataset_1[(dataset_1.current_load>=0.006)].reset_index(drop=True)

# converting time seconds->hour
df_1["time"]=df_1["time"]/3600

#calculate Soc for each cycle
cycle_count=df_1["cycle"].unique()
Soc_list=[]

# Update Soc for each reading within each cycle using Coulomb counting
for cycle in cycle_count:
    cycle_data = df_1[df_1['cycle'] == cycle]
    
    cumulative_Soc = [sum(cycle_data["capacity"])/len(cycle_data["capacity"])]
    
    for i in range(1, len(cycle_data)):
        
        delta_time = cycle_data['time'].iloc[i-1] - cycle_data['time'].iloc[i]
        
        delta_charge = cycle_data['current'].iloc[i] * delta_time
        
        cumulative_Soc.append(cumulative_Soc[-1] -delta_charge)
        
    Soc_list.extend(cumulative_Soc)

#DOD is the opposite of the soc 
df_1["Soc"] = [(q / C_max) for q in Soc_list]
df_1["Soh"]=df_1["capacity"]/C_max

# selecting parameters 
att_=["cycle","capacity","voltage","current","temperature","current_load","voltage_load","time","Soc"]
att=["cycle","current_load","voltage_load","Soc","time"]
X_axis="cycle"

#seperating dataset
X_train = df_1[att]
y_train = df_1[["Soh"]]



#renaming columns for simplicity
dataset_2.columns=['cycle',"capacity",'voltage', 'current','temperature', 'current_load', 'voltage_load',"time"]
dataset_2["current_load"]= abs(dataset_2["current_load"])

#removing ideling / starting and ending current level data  
df_2=dataset_2[(dataset_2.current_load>=0.006)].reset_index(drop=True)

# converting time seconds->hour
df_2["time"]=df_2["time"]/3600

#calculate Soc for each cycle
cycle_count=df_2["cycle"].unique()
Soc_list=[]

# Update Soc for each reading within each cycle using Coulomb counting
for cycle in cycle_count:
    cycle_data = df_2[df_2['cycle'] == cycle]
    
    cumulative_Soc = [sum(cycle_data["capacity"])/len(cycle_data["capacity"])]
    
    for i in range(1, len(cycle_data)):
        
        delta_time = cycle_data['time'].iloc[i-1] - cycle_data['time'].iloc[i]
        
        delta_charge = cycle_data['current'].iloc[i] * delta_time
        
        cumulative_Soc.append(cumulative_Soc[-1] -delta_charge)
        
    Soc_list.extend(cumulative_Soc)

#DOD is the opposite of the soc 
df_2["Soc"] = [(q / C_max) for q in Soc_list]
df_2["Soh"]=df_2["capacity"]/C_max

X_test = df_2[att]
y_test = df_2[["Soh"]]

# Training models

In [4]:
# random forest model
regressor1 = RandomForestRegressor(n_estimators=100,random_state=0)
regressor1.fit(X_train,y_train.values.ravel())

# polynomial model

poly = PolynomialFeatures(degree=1)  # Choose the degree based on your problem
X_poly = poly.fit_transform(X_train)
poly_reg_model = LinearRegression()
poly_reg_model.fit(X_poly, y_train)

# lstm model

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Reshape data for LSTM (samples, timesteps, features)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))

# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(25, activation='tanh', recurrent_activation='sigmoid', return_sequences=True,input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(25, return_sequences=True))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(25, return_sequences=True))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(25))
lstm_model.add(Dropout(0.25))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lstm_model.fit(X_train_lstm, y_train, epochs=200,batch_size=32, verbose=1,validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1129/1129 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.1191 - val_loss: 6.3851e-05
Epoch 2/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0065 - val_loss: 2.1080e-04
Epoch 3/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0039 - val_loss: 2.0388e-04
Epoch 4/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0021 - val_loss: 2.2073e-04
Epoch 5/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 9.6525e-04 - val_loss: 2.5266e-04
Epoch 6/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4.4970e-04 - val_loss: 5.8958e-04
Epoch 7/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.9004e-04 - val_loss: 3.9331e-04
Epoch 8/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.4692e-04 - val_loss: 5.7519e-04
Epoch 9/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2.4366e-04 - val_loss: 5.6871e-04
Epoch 10/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.3246e-04 - val_loss: 5.4846e-04
Epoch 11/200
1129/1129 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step -

In [5]:
# making predictions

# poly_prediction
X_test_poly = poly.transform(X_test)
poly_predictions = poly_reg_model.predict(X_test_poly)

# LSTM Predictions
X_test_scaled = scaler.transform(X_test)
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))
lstm_predictions = lstm_model.predict(X_test_lstm)

# rfr Prediction
rfr_predictions = regressor1.predict(X_test)

1387/1387 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


In [6]:
poly_predictions.ravel()

array([0.99674387, 0.99676783, 0.99677593, ..., 0.57950412, 0.57955365,
       0.57961282])

In [7]:
lstm_predictions.ravel()

array([0.912102  , 0.91146994, 0.9110232 , ..., 0.5789057 , 0.57817584,
       0.5773214 ], dtype=float32)

In [8]:
rfr_predictions = rfr_predictions.reshape(len(rfr_predictions),1)
rfr_predictions

array([[0.90560362],
       [0.90560362],
       [0.90560362],
       ...,
       [0.64638016],
       [0.64638016],
       [0.64638016]])

# stacking prediction Poly+LSTM

In [12]:
# Calculate performance weighted average

rmse_poly = np.sqrt(metrics.mean_squared_error(y_test, rfr_predictions))
rmse_lstm = np.sqrt(metrics.mean_squared_error(y_test, lstm_predictions))

# Assign weights inversely proportional to MSE

weight_poly = 1 / rmse_poly
weight_lstm_p = 1 / rmse_lstm
total_weight_p = weight_poly + weight_lstm_p
weight_poly /= total_weight_p
weight_lstm_p /= total_weight_p

# Combine predictions
Poly_Lstm = (weight_poly * poly_predictions + weight_lstm_p * lstm_predictions)

# stacking prediction RFR+LSTM

In [13]:
# Calculate performance weighted average

rmse_rfr = np.sqrt(metrics.mean_squared_error(y_test, poly_predictions))

# Assign weights inversely proportional to MSE

weight_rfr = 1 / rmse_poly
weight_lstm_r = 1 / rmse_lstm
total_weight_r = weight_rfr + weight_lstm_r
weight_rfr /= total_weight_r
weight_lstm_r /= total_weight_r

# Combine predictions
Rfr_Lstm = (weight_rfr * rfr_predictions + weight_lstm_r * lstm_predictions)

# graph

In [28]:
from mpl_toolkits.mplot3d import Axes3D

In [31]:
#set graph
ax = plt.figure(figsize=(12,6),dpi=150).add_subplot(111, projection='3d')

#plot data
plt.plot(X_test["cycle"],y_test,color='red',label='actual',linewidth=2)
plt.plot(X_test["cycle"],Poly_Lstm,color='blue',label="lstm+RFR_100",linewidth=2)
plt.plot(X_test["cycle"],Rfr_Lstm,color='green',label="lstm+RFR_100",linewidth=2)
plt.plot(X_test["cycle"],rfr_predictions,color='yellow',label="lstm+RFR_100",linewidth=2)
plt.plot(X_test["cycle"],poly_predictions,color='Purple',label="lstm+RFR_100",linewidth=2)


# graph properties

ax.set_xlabel("Cycle")
ax.set_ylabel("Method")
ax.set_zlabel("SOH")
ax.set_yticks([0, 1, 2, 3, 4])
ax.set_yticklabels(['actual', 'Poly+Lstm', 'Rfr+lstm', 'Rfr', 'Poly'])
ax.set_title("Comparison Graph")

plt.legend(loc="upper right")
plt.show()

: 